In [ ]:
import wandb
wandb.login()

In [ ]:
import transformers
import torch
print(transformers.__version__)
print(torch.__version__)

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.46.3


In [1]:
import torch
print("GPU 사용 가능 여부:", torch.cuda.is_available())

GPU 사용 가능 여부: True


In [2]:
from transformers import T5Tokenizer, T5ForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.metrics import accuracy_score, f1_score
from torch.utils.data import Dataset

from sklearn.model_selection import train_test_split

import pandas as pd
import torch

## focal loss 

In [3]:
## dataset 불러오기
final_data = pd.read_csv('final_train_2.csv')

In [4]:
data = final_data.copy()

In [5]:
# label 분포 확인
data['loan_approved'].value_counts()

loan_approved
0    4810
1    2647
Name: count, dtype: int64

모델을 저장 할 PATH설정

In [ ]:
MODEL_PATH = 'tmp_model_5' # 저장경로

##### Custom Dataset 정의

In [6]:
class CustomDataset(Dataset):
    def __init__(self, tokenizer, texts, labels, max_length=512):
        self.tokenizer = tokenizer
        self.texts = texts
        self.labels = labels
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        inputs = self.tokenizer(
            self.texts[idx],
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
            )
        inputs['input_ids'] = inputs['input_ids'].squeeze()
        inputs['attention_mask'] = inputs['attention_mask'].squeeze()
        inputs['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return inputs

##### 데이터 불러오기 및 Train, Validation set으로 쪼개기

In [7]:

final_data_df = data.copy()

reasoning_prompt = final_data_df['reasoning_sentences'].tolist() # reasoning 리스트
loan_approved = final_data_df['loan_approved'].tolist() # rating 리스트

print(f'reasoning_prompt 개수 : {len(reasoning_prompt)}')
print(f'대출 승인여부 개수 : {len(loan_approved)}')


# 데이터셋 분리
train_sentences, valid_sentences, train_scores, valid_scores = train_test_split(reasoning_prompt,
                                                                                loan_approved,
                                                                                test_size=0.2,
                                                                                random_state=42, stratify = loan_approved)

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
model = T5ForSequenceClassification.from_pretrained("google/flan-t5-small", num_labels=2)


train_dataset = CustomDataset(tokenizer,
                              train_sentences,
                              train_scores,
                              max_length=256)

valid_dataset = CustomDataset(tokenizer,
                              valid_sentences,
                              valid_scores,
                              max_length=256)

reasoning_prompt 개수 : 7457
대출 승인여부 개수 : 7457


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at google/flan-t5-small and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


##### 평가 지표 정의(accuracy, f1)

In [37]:
def compute_metrics(pred):
    labels = pred.label_ids
    predictions = pred.predictions[0] if isinstance(pred.predictions, tuple) else pred.predictions
    preds = predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")  # F1-score
    return {"accuracy": acc, "f1": f1}

#### training args 정의

In [ ]:
training_args = TrainingArguments(
    output_dir='./result',
    overwrite_output_dir=True,
    logging_dir='./logs',
    logging_steps=10,
    num_train_epochs=50,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    save_strategy='epoch',
    eval_strategy='epoch',
    load_best_model_at_end=True, # 최적의 모델 로드
    metric_for_best_model="eval_loss", # Early Stopping 기준 메트릭
    greater_is_better=False,     # 평가 메트릭이 작을수록 더 나은 경우 (e.g., loss)
    save_total_limit=1,
    learning_rate=1e-5,
    weight_decay=0.01,
    warmup_steps=100,
    report_to='wandb',
    run_name='mini_project_2_loan_approved',
    seed=42
)

##### 학습

In [39]:
import torch
import torch.nn.functional as F

class FocalLoss(torch.nn.Module):
    def __init__(self, alpha=1.0, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)  # pt = probability of correct class
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss


In [ ]:
from transformers import Trainer

class FocalLossTrainer(Trainer):
    def __init__(self, focal_loss_params, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.loss_fct = FocalLoss(**focal_loss_params) if focal_loss_params else nn.CrossEntropyLoss()


    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        # loss_fct를 사용하여 손실 계산
        loss = self.loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss



In [ ]:
## Custom trainer 정의 
from torch import nn
from transformers import Trainer

# CustomTrainer 사용
focal_loss_params = {"alpha": 0.25, "gamma": 2.0, "reduction": "mean"}
trainer = FocalLossTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
    focal_loss_params=focal_loss_params ,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

In [ ]:
trainer.train()
trainer.evaluate()

model.save_pretrained(MODEL_PATH)
tokenizer.save_pretrained(MODEL_PATH)

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.044500,0.041391,0.645442,0.510675
2,0.042500,0.040516,0.645442,0.510675
3,0.041400,0.039853,0.646783,0.513705
4,0.040600,0.039072,0.648794,0.521635
5,0.039800,0.038133,0.661528,0.559479
6,0.038600,0.036958,0.698391,0.645144
7,0.037700,0.035765,0.714477,0.679806
8,0.035600,0.035649,0.723190,0.692378
9,0.034400,0.035261,0.735255,0.718371
10,0.034000,0.034908,0.733244,0.719541


There were missing keys in the checkpoint model loaded: ['transformer.encoder.embed_tokens.weight', 'transformer.decoder.embed_tokens.weight'].


('tmp_model_5/tokenizer_config.json',
 'tmp_model_5/special_tokens_map.json',
 'tmp_model_5/spiece.model',
 'tmp_model_5/added_tokens.json')